In [1]:
%pip install -qU langchain_community jq 
%pip install -qU langchain-openai
%pip install -qU langchain-core
%pip install -qU langchain-mongodb pymongo
%pip install openai


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from langchain_community.document_loaders import JSONLoader
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
# from langchain_text_splitters import RecursiveJsonSplitter


In [3]:
LLMKey = os.getenv('LLMKey')
LangChainKey = os.getenv('LangchainKey')

### JSON Loader by Langchain
Using langchain's library to easily and automatically split data into chunks in preparation for embedding. 

In [4]:
loader = JSONLoader(
    file_path="./dataset/ReviewCollection.json",
    jq_schema=".[]",
    text_content=False,
)

reviews = loader.load()
print(reviews[0])

page_content='{"id": 20001, "app_id": "com.whatsapp", "name": "Alii Said", "rating": 5, "content": "log in whatsapp", "date": "2025-05-24 18:55:57", "topic_id": null, "sentiment_score": 0}' metadata={'source': '/Users/walido/Documents/Projects/RAG Pipeline Supported Chatbot/dataset/ReviewCollection.json', 'seq_num': 1}


### Preparing Embeddings by OpenAI
Embedding is the process of converting textual data into its numerical representation. 

In [5]:
embeddings = OpenAIEmbeddings(openai_api_key=LLMKey, model="text-embedding-3-small")

### Utilizing In-memory Vector Store
Database responsible for storing embeddings and performing similarity searches.

In [6]:
vector_store = InMemoryVectorStore(embeddings)


In [7]:
vector_store.add_documents(documents=reviews)

['86e60570-93d4-4ca3-94a2-c6eab80492be',
 '197f26de-2a88-498c-ae55-844ffc956e7f',
 '8479d96f-9240-4f16-bdcb-7dab9a330895',
 'fe7b8415-378a-47d8-b336-16f560c18421',
 '6423c3ec-17e7-4a63-bbcc-cdd8693b0630',
 '6708dc57-a452-459b-980f-926b3deb4b2b',
 'b2522252-a428-489a-996f-4357244712ea',
 '8159049b-6026-4c55-9c15-c9b1a2af4584',
 '00b1c2b3-0094-4238-ad81-206d4123a58b',
 'd278cb97-3b3e-4ee4-8f3d-7b82319b9b6e',
 '1eb1d972-773e-4de3-b8b8-c578039e3896',
 '0b4af48d-9c70-43ba-90ae-87269ea18631',
 'ff7b29fa-4947-4dfc-9956-bab1014e593b',
 'a7fca608-8100-4e20-9acf-f87d6f67af44',
 '24daac24-4502-4889-9a7d-6d7b9f458f5e',
 'cdfb6a86-3402-4df8-8260-b31074e11cff',
 'bf5010ca-2265-458d-98b3-5d1dc10e47f4',
 '16b001f5-0d23-47b2-99d2-57971f50bf7d',
 '0edc9283-ded4-41d2-bfdc-da4dc126753d',
 'ae126893-ebce-4b77-a3c2-6bd652faecfc',
 '689d60e1-87f6-4f4d-9ff5-7b7f30f0c238',
 'd541463c-dbaa-4875-b5f4-e742a72856bd',
 'cce35ee6-a66d-4212-a414-cc2d7046186d',
 '1ae02441-81f3-4469-87c7-5fbe3efc81f3',
 'cdaaff9f-08d9-

Function that would find the suitable embeddings relevant to the query.

In [8]:
def searchVector(query):
    results = vector_store.similarity_search(query=query)
    return "\n".join(result.page_content for result in results)

### Retrieval and Augmentation

In [11]:
from openai import OpenAI
import json

query = "What are some of the most common recent problems that users face?"
client = OpenAI(api_key=LLMKey)

In [15]:
def retrieve(query):
    docs = vector_store.similarity_search(query, k=30)
    return "\n".join(doc.page_content for doc in docs)

In [16]:
context = retrieve(query)

with open('dataset/AppInfo.json', 'r') as file:
    appInfo = json.load(file)

template = f"""
You are a product manager assistant in charge of answering questions about an application. 
You will be given the application details, and a context containing relevant application reviews. 
Answer your questions mainly off the details from the reviews that you will be given. If you make any assumptions,
clearly mention that they are assumptions. 
App Info: {appInfo}
Context: {context}
Query: {query}
"""
print(template)
response = client.responses.create(
    model="o4-mini",
    input = template
)
print(response.output_text)


You are a product manager assistant in charge of answering questions about an application. 
You will be given the application details, and a context containing relevant application reviews. 
Answer your questions mainly off the details from the reviews that you will be given. If you make any assumptions,
clearly mention that they are assumptions. 
App Info: [{'id': 'com.whatsapp', 'name': 'WhatsApp Messenger', 'description': 'WhatsApp from Meta is a FREE messaging and video calling app. It’s used by over 2B people in more than 180 countries. It’s simple, reliable, and private, so you can easily keep in touch with your friends and family. WhatsApp works across mobile and desktop even on slow connections, with no subscription fees*.\r\n\r\nPrivate messaging across the world\r\n\r\nYour personal messages and calls to friends and family are end-to-end encrypted. No one outside of your chats, not even WhatsApp, can read or listen to them.\r\n\r\nSimple and secure connections, right away\r\